<a href="https://colab.research.google.com/github/estudos-wanderson/Modelagem-Matem-tica/blob/main/Tema_3_Sistemas_de_Equa%C3%A7%C3%B5es_Lineares_e_Ajuste_de_Curvas_em_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Método de eliminação de Gauss**

### **Processo de Eliminação de Gauss**

Considere um sistema de equações lineares $AX=B$ com $n$ equações e $n$ incógnitas. O objetivo é transformar a matriz de coeficientes $A$ em uma matriz triangular superior $U$ zerando todos os elementos abaixo da diagonal principal.

1. Passo 1 (para a coluna 1)
    - Escolha o primeiro elemento da primeira linha, $a_{11}$, como pivô.
    - Se $a_{11}=0$, troque a primeira linha com uma linha abaixo onde o elemento correspondente na coluna 1 seja não nulo. Se todos os elementos da primeira coluna forem zero, o processo de eliminação para esta coluna não é possível, então prossiga para a próxima coluna.
    - Para cada linha $i$ abaixo da primeira (para $i=2, \dots, n$), calcule o multiplicador $m_{i1} = \frac{a_{i1}}{a_{11}}$.
    - Substitua a linha $L_i$ por $L_i - m_{i1}L_1$. Este processo zera o elemento $a_{i1}$.
    - Repita o processo para todas as linhas abaixo da primeira.

2. Passo k (para a coluna k)
    - Ignore as $k-1$ primeiras linhas e colunas.
    - Considere o elemento $a_{kk}$ da matriz transformada como o novo pivô.
    - Se $a_{kk}=0$, troque a linha $k$ com uma linha abaixo que tenha um elemento não nulo na coluna $k$.
    - Para cada linha $i$ abaixo da linha $k$ (para $i=k+1, \dots, n$), calcule o multiplicador $m_{ik} = \frac{a_{ik}}{a_{kk}}$.
    - Substitua a linha $L_i$ por $L_i - m_{ik}L_k$.
    - Repita este processo até que a matriz $A$ se torne uma matriz triangular superior $U$.

O sistema original $AX=B$ é transformado no sistema triangular superior $UX=C$, onde $C$ é a coluna dos termos independentes $B$ transformada. O sistema final terá a forma:

$$
\begin{pmatrix}
u_{11} & u_{12} & \cdots & u_{1n} \\
0 & u_{22} & \cdots & u_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & u_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}
=
\begin{pmatrix}
c_1 \\
c_2 \\
\vdots \\
c_n
\end{pmatrix}
$$

3. Retrossubstituição

Após a eliminação de Gauss, o sistema de equações lineares é transformado em um sistema triangular superior $UX=C$, que é mais simples de resolver. O processo de retrossubstituição permite encontrar os valores das incógnitas a partir da última equação, "subindo" e substituindo os valores já encontrados.

- *Passo 1*: Resolva a última equação para $x_n$.
$$x_n = \frac{c_n}{u_{nn}}$$
- *Passo 2*: Substitua o valor de $x_n$ na penúltima equação e resolva para $x_{n-1}$.
- *Passo 3*: Continue "subindo" e substituindo os valores já encontrados nas equações anteriores, até resolver para $x_1$.

A fórmula geral para a retrossubstituição é:

Para $i = n-1, n-2, \dots, 1$:
$$x_i = \frac{1}{u_{ii}} \left( c_i - \sum_{j=i+1}^{n} u_{ij}x_j \right)$$

In [29]:
import numpy as np

def change_rows(A, i, j):
    A[[i, j]] = A[[j, i]]

def search_pivot_row(A, k):
    for i in range(k+1, len(A)):
        if A[i, k] != 0:
            return i
    return None

def gauss_elimination(A, B):
    n = len(B)
    if A.shape[0] != n or A.shape[1] != n:
        raise ValueError("A and B must have the same number of rows")

    for k in range(n - 1):
        if A[k, k] == 0:
            pivot_row = search_pivot_row(A, k)
            if pivot_row is None:
                raise ValueError("No unique solution exists")
            change_rows(A, k, pivot_row)
            change_rows(B, k, pivot_row)

        for i in range(k + 1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] = A[i, k:] - m * A[k, k:]
            B[i] = B[i] - m * B[k]

    if A[n - 1, n - 1] == 0:
        raise ValueError("No unique solution exists")

    x = np.zeros(n)
    x[n - 1] = B[n - 1] / A[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        sum = 0
        for j in range(i + 1, n):
            sum += A[i, j] * x[j]
        x[i] = 1 / A[i, i] * (B[i] - sum)

    return x

Exemplo

In [30]:
A = np.array([[1, 2, 3], [4, 7, 6], [5, 8, 6]])
B = np.array([1, 2, 3])

sol = gauss_elimination(A, B)
print(sol)

[ 3.         -2.          0.66666667]


### **Processo de Eliminação de Gauss-Jordan**

O método de Eliminação de Gauss-Jordan é uma variação do método de Gauss que transforma a matriz de coeficientes $A$ em uma **matriz diagonal identidade** $I$, em vez de uma matriz triangular superior $U$. Isso elimina a necessidade da retrossubstituição, pois a solução do sistema de equações é obtida diretamente.

1.  **Passo 1 (para a coluna 1)**
    * Escolha o primeiro elemento da primeira linha, $a_{11}$, como pivô.
    * Se $a_{11}=0$, troque a primeira linha com uma linha abaixo onde o elemento correspondente na coluna 1 seja não nulo. Se todos os elementos da primeira coluna forem zero, o processo de eliminação não é possível, então prossiga para a próxima coluna.
    * Para normalizar o pivô, divida a primeira linha por $a_{11}$, tornando o novo pivô igual a 1.
    * Para cada linha $i$ (para $i=2, \dots, n$), calcule o multiplicador $m_{i1} = a_{i1}$.
    * Substitua a linha $L_i$ por $L_i - m_{i1}L_1$. Este processo zera todos os elementos abaixo do pivô na coluna 1.

2.  **Passo k (para a coluna k)**
    * Considere o elemento $a_{kk}$ da matriz transformada como o novo pivô.
    * Se $a_{kk}=0$, troque a linha $k$ com uma linha abaixo que tenha um elemento não nulo na coluna $k$.
    * Para normalizar o pivô, divida a linha $k$ por $a_{kk}$, tornando o novo pivô igual a 1.
    * Para cada linha $i$ (para $i=1, \dots, n$ e $i \ne k$), calcule o multiplicador $m_{ik} = a_{ik}$.
    * Substitua a linha $L_i$ por $L_i - m_{ik}L_k$. Este processo zera todos os elementos acima e abaixo do pivô na coluna $k$.
    * Repita este processo até que a matriz $A$ se torne a matriz identidade $I$.

O sistema original $AX=B$ é transformado no sistema diagonal $IX=D$, onde $D$ é a coluna dos termos independentes $B$ transformada. O sistema final terá a forma:

$$
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}
=
\begin{pmatrix}
d_1 \\
d_2 \\
\vdots \\
d_n
\end{pmatrix}
$$

A solução é obtida diretamente, pois $x_i = d_i$ para $i=1, 2, \dots, n$.